In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, MaxPool2D,Dense,InputLayer,Flatten,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError,BinaryAccuracy,TruePositives,FalsePositives,TrueNegatives,FalseNegatives,Precision,Recall,AUC
from tensorflow.keras.losses import MeanSquaredError,Huber,MeanAbsoluteError,BinaryCrossentropy

In [ ]:
dataset,dataset_info = tfds.load('malaria',with_info=True,as_supervised=True,shuffle_files=True,split=['train'])

In [ ]:
TRAIN_RATIO = 0.8
TEST_RATIO = 0.1
VAL_RATIO = 0.1

In [ ]:
def split(dataset,TRAIN_RATIO,TEST_RATIO,VAL_RATIO):

  DATASET_SIZE = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO * DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO * DATASET_SIZE))


  val_dataset = val_test_dataset.take(int(VAL_RATIO * DATASET_SIZE))


  test_dataset = val_test_dataset.skip(int(VAL_RATIO * DATASET_SIZE))

  return train_dataset,test_dataset,val_dataset




In [ ]:
train_dataset,test_dataset,val_dataset = split(dataset[0],TRAIN_RATIO,TEST_RATIO,VAL_RATIO)
print(list(train_dataset.take(1).as_numpy_iterator()))
print(list(test_dataset.take(1).as_numpy_iterator()))
print(list(val_dataset.take(1).as_numpy_iterator()))

Data Visualization

In [ ]:
for i,(image,label) in enumerate(train_dataset.take(16)):
  ax = plt.subplot(4,4,i+1)
  plt.imshow(image)
  plt.title(dataset_info.features['label'].int2str(label))
  plt.axis('off')

In [ ]:
dataset_info.features['label'].int2str(0)

Data Preprocessing


In [ ]:
IM_SIZE = 224
def resize_rescale(image,label):
  return tf.image.resize(image,(IM_SIZE,IM_SIZE))/255.0,label

In [ ]:
train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

In [ ]:
for image,label in train_dataset.take(1):
  print(image,label)

In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=8,reshuffle_each_iteration=True,).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:

val_dataset = val_dataset.shuffle(buffer_size=8,reshuffle_each_iteration=True,).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
val_dataset

In [ ]:
train_dataset

In [ ]:
model = tf.keras.Sequential([
    InputLayer(input_shape=(IM_SIZE,IM_SIZE,3)),
    Conv2D(filters = 6, kernel_size = 3, strides=1,padding='valid',activation='relu'),
     BatchNormalization(),
    MaxPool2D(pool_size=2,strides=1),



    Conv2D(filters = 16, kernel_size = 5, strides=1,padding='valid',activation='relu'),
     BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2),
    Flatten(),
    Dense(100, activation = "relu"),
    Dense(10, activation = "relu"),
    Dense(1, activation = "sigmoid"),

])

model.summary()




In [ ]:
model.compile(optimizer=Adam(learning_rate=0.1),
              loss = BinaryCrossentropy(),
              metrics = ['accuracy'])


In [ ]:
history = model.fit(train_dataset, validation_data = val_dataset,epochs = 20, verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model_loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['train_loss','val_loss'],loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model_Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train_accuracy','val_accuracy'],loc='upper right')
plt.show()

Model Evaluation and Testing

In [ ]:
test_dataset = test_dataset.batch(1)

In [ ]:
model.evaluate(test_dataset)

In [ ]:
def parasite_or_not(x):
  if(x<0.5):
    return str("P")
  else:
    return str("U")

In [ ]:
parasite_or_not(model.predict(test_dataset.take(1))[0][0])

In [ ]:
for i,(image,label) in enumerate(test_dataset.take(9)):
  ax = plt.subplot(3,3,i+1)
  plt.imshow(image[0])
  plt.title(str(parasite_or_not(label.numpy()[0])) + ":" + str(parasite_or_not(model.predict(image)[0][0])))
  plt.axis('off')

In [ ]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Layer


In [ ]:
func_input = Input(shape=(IM_SIZE,IM_SIZE,3),name = 'Input Image')

x = Conv2D(filters = 6, kernel_size = 3, strides=1,padding='valid',activation='relu')(func_input)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=2,strides=1)(x)

x = Conv2D(filters = 16, kernel_size = 5, strides=1,padding='valid',activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=2,strides=2)(x)


output = Dense(1, activation = "sigmoid")(x)


feature_extractor_model = Model(func_input,output,name = "feature_extractor")

feature_extractor_model.summary()



In [ ]:
feature_extractor_sequential_model = tf.keras.Sequential([
    InputLayer(input_shape=(IM_SIZE,IM_SIZE,3)),
    Conv2D(filters = 6, kernel_size = 3, strides=1,padding='valid',activation='relu'),
     BatchNormalization(),
    MaxPool2D(pool_size=2,strides=1),



    Conv2D(filters = 16, kernel_size = 5, strides=1,padding='valid',activation='relu'),
     BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2),
    
])

feature_extractor_sequential_model.summary()




In [ ]:
func_input = Input(shape=(IM_SIZE,IM_SIZE,3),name = 'Input Image')

x = feature_extractor_model(func_input)

x = Flatten()(x)

x = Dense(100, activation = "relu")(x)
x = BatchNormalization()(x)

x = Dense(10, activation = "relu")(x)
x = BatchNormalization()(x)

func_output = Dense(1, activation = "sigmoid")(x)


lenet_model_func = Model(func_input,func_output, name = 'Lenet Model')

lenet_model_func.summary()



In [ ]:
func_input = Input(shape=(IM_SIZE,IM_SIZE,3),name = 'Input Image')

x = feature_extractor_model(func_input)

x = Flatten()(x)

x = Dense(100, activation = "relu")(x)
x = BatchNormalization()(x)

x = Dense(10, activation = "relu")(x)
x = BatchNormalization()(x)

func_output = Dense(1, activation = "sigmoid")(x)


lenet_model_func = Model(func_input,func_output, name = 'Lenet Model')

lenet_model_func.summary()



In [ ]:
metrics = [TruePositives(name="tp"),FalsePositives(name="fp"),TrueNegatives(name="tn"),FalseNegatives(name="fn"),BinaryAccuracy(name="accuracy"),Precision(name="precision"),Recall(name="recall"),AUC(name="auc")]

In [ ]:
lenet_model_func.compile(optimizer=Adam(learning_rate=0.1),
              loss = BinaryCrossentropy(),
              metrics = metrics)


In [ ]:
history = lenet_model_func.fit(train_dataset, validation_data = val_dataset,epochs = 20, verbose=1)

Model Subclassing

In [ ]:
class FeatureExtractor(Layer):
    def __init__(self, filters, kernel_size, strides, padding, activation, pool_size):
        super(FeatureExtractor, self).__init__()

        self.conv2d_1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, 
                               padding=padding, activation=activation)
        self.batch_1 = BatchNormalization()
        self.pool_1 = MaxPool2D(pool_size=pool_size, strides=2*strides)

        self.conv2d_2 = Conv2D(filters=filters * 2, kernel_size=kernel_size, strides=strides, 
                               padding=padding, activation=activation)
        self.batch_2 = BatchNormalization()
        self.pool_2 = MaxPool2D(pool_size=pool_size, strides=2*strides)

    def call(self, x):
        x = self.conv2d_1(x)
        x = self.batch_1(x)
        x = self.pool_1(x)

        x = self.conv2d_2(x)
        x = self.batch_2(x)
        x = self.pool_2(x)

        return x

feature_sub_classed = FeatureExtractor(8,3,1,'valid','relu',2)


In [ ]:
func_input = Input(shape=(IM_SIZE,IM_SIZE,3),name = 'Input Image')

x = feature_sub_classed(func_input)

x = Flatten()(x)

x = Dense(100, activation = "relu")(x)
x = BatchNormalization()(x)

x = Dense(10, activation = "relu")(x)
x = BatchNormalization()(x)

func_output = Dense(1, activation = "sigmoid")(x)


lenet_model_func = Model(func_input,func_output, name = 'Lenet Model')

lenet_model_func.summary()



In [ ]:
class LenetModel(Model):
    def __init__(self):
        super(LenetModel, self).__init__()

        
        self.feature_extractor = FeatureExtractor(8, 3, 1, "valid", "relu", 2)
        self.flatten = Flatten()
        self.dense_1 = Dense(100, activation="relu")
        self.batch_1 = BatchNormalization()
        self.dense_2 = Dense(10, activation="relu")
        self.batch_2 = BatchNormalization()
        self.dense_3 = Dense(1, activation="sigmoid")

    def call(self, x, training=False):
        x = self.feature_extractor(x, training=training)
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.batch_1(x, training=training)
        x = self.dense_2(x)
        x = self.batch_2(x, training=training)
        return self.dense_3(x)


lenet_sub_classed = LenetModel() 
lenet_sub_classed(tf.zeros((1, 224, 224, 3))) 
lenet_sub_classed.summary()


In [ ]:
lenet_sub_classed.compile(optimizer=Adam(learning_rate=0.1),
              loss = BinaryCrossentropy(),
              metrics = ['accuracy'])

In [ ]:
history = lenet_sub_classed.fit(train_dataset, validation_data = val_dataset,epochs = 20, verbose=1)  

In [ ]:
class MyDense(Layer):
    def __init__(self, output_units):
        super(MyDense, self).__init__()
        self.output_units = output_units

    def build(self, input_shape):
    
        input_dim = input_shape[-1]
        
      
        self.w = self.add_weight(
            shape=(input_dim, self.output_units),
            initializer='random_normal',
            trainable=True,
            name="weights"
        )
        self.b = self.add_weight(
            shape=(self.output_units,),
            initializer='zeros',
            trainable=True,
            name="bias"
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


In [ ]:
from tensorflow.keras.layers import Activation

IM_SIZE = 224
func_input = Input(shape=(IM_SIZE, IM_SIZE, 3), name='Input Image')

x = feature_sub_classed(func_input)

x = Flatten()(x)

x = MyDense(100)(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = MyDense(10)(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

func_output = Dense(1, activation='sigmoid')(x)  # Keep this as-is

lenet_model_func = Model(func_input, func_output, name='Lenet Model')

lenet_model_func.summary()